In [1]:
from keras.applications.vgg16 import VGG16
from keras.models import Sequential,Model 
from keras.layers import Dense,Dropout,Activation,Flatten,GlobalAveragePooling2D
from keras.layers import Conv2D,MaxPooling2D,ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K

In [2]:
img_rows,img_cols = 256,256

VGG = VGG16(weights='imagenet',include_top=False,input_shape=(img_rows,img_cols,3))

for layer in VGG.layers:
    layer.trainable=False

def addTopModelVGG(bottom_model, num_classes):

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(256,activation='relu')(top_model)
    top_model = Dense(256,activation='relu')(top_model)
    top_model = Dense(128,activation='relu')(top_model)
    top_model = Dense(128,activation='relu')(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)

    return top_model

num_classes = 20

FC_Head = addTopModelVGG(VGG,num_classes)

model = Model(inputs = VGG.input, outputs = FC_Head)

print(model.summary())

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)      

In [3]:
def recall(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Negative) = 실제 값이 1(Positive) 전체
    count_true_positive_false_negative = K.sum(y_target_yn)

    # Recall =  (True Positive) / (True Positive + False Negative)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    recall = count_true_positive / (count_true_positive_false_negative + K.epsilon())

    # return a single tensor value
    return recall


def precision(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Positive) = 예측 값이 1(Positive) 전체
    count_true_positive_false_positive = K.sum(y_pred_yn)

    # Precision = (True Positive) / (True Positive + False Positive)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    precision = count_true_positive / (count_true_positive_false_positive + K.epsilon())

    # return a single tensor value
    return precision


def f1score(y_target, y_pred):
    _recall = recall(y_target, y_pred)
    _precision = precision(y_target, y_pred)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    _f1score = ( 2 * _recall * _precision) / (_recall + _precision+ K.epsilon())
    
    # return a single tensor value
    return _f1score

In [ ]:
train_data_dir = 'C:/Users/piai/Desktop/Contest/NIPA/train'
validation_data_dir = 'C:/Users/piai/Desktop/Contest/NIPA/val'


train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=30,
                    width_shift_range=0.3,
                    height_shift_range=0.3,
                    horizontal_flip=True,
                    fill_mode='nearest'
                                   )

validation_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 32

############################################ flow_from_directory ###############################################
# flow_from_directory 메소드 : 폴더구조를 그대로 가져와서 ImageDataGenerator 객체의 실제 데이터를 채워줌.
# 이 데이터를 불러올 때 앞서 정의한 파라미터로 전처리 함.
# 마지막으로 밑에서 fit_generator 함수를 실행함으로써 fitting이 이루어짐.
train_generator = train_datagen.flow_from_directory(
                        train_data_dir,
                        target_size = (img_rows,img_cols),
                        batch_size = batch_size,
                        class_mode = 'categorical'
                        )

validation_generator = validation_datagen.flow_from_directory(
                            validation_data_dir,
                            target_size=(img_rows,img_cols),
                            batch_size=batch_size,
                            class_mode='categorical')


from keras.optimizers import RMSprop,Adam
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau


########################################### ModelCheckpoint ###############################################
# keras의 콜백함수인 ModelCheckpoint
# 모델이 학습하면서 정의한 조건을 만족했을 때 모델의 weight값을 중간 저장해 줌.
# 모델이 개선된 validation score를 도출해낼 때마다 weight를 중간 저장함으로써 중간에 끊겨도 다시 weight를 불러와 학습 이어나갈 수 있음.

checkpoint = ModelCheckpoint(
                             'vgg_16.h5',
                             monitor='val_loss',       # val_loss 값이 개선되었을 때 호출
                             mode='min',               # best값을 찾을 때 min으로 찾기( auto/ min / max)
                             save_best_only=True,     # 가장 best 값만 저장
                             verbose=1)                # 로그 출력

########################################## EarlyStopping ####################################################
# 콜백함수
# model의 성능 지표가 설정한 epoch동안 개선되지 않을 때 조기 종료
# ModelCheckpoint 콜백의 조합을 통하여, 개선되지 않는 학습에 대한 조기 종료를 실행하고,
# ModelCheckpoint로 부터 가장 best model을 다시 로드하여 학습을 재게할 수 있음
earlystop = EarlyStopping(
                          monitor='val_loss',                 # 모니터 기준 설정 (val loss)
                          min_delta=0,
                          restore_best_weights=True,
                          patience=10,                        # 10회 epoch 동안 개선되지 않는다면 종료
                          verbose=1)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=5, 
                                            verbose=1, 
                                            factor=0.2, 
                                            min_lr=0.0001)

# 콜백 리스트에 ModelCheckpoint와 EsrlyStopping 모두 넘겨줌으로써 best model을 중간 저장함과 동시에
# 더 이상 개선되지 않을 때 조기 종료시킬 수 있음.
callbacks = [earlystop,checkpoint,learning_rate_reduction]  

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy']#, precision, recall, f1score] ==> 학습할 때 확인은 가능하나, 이거 넣은 모델로 .h5파일 뽑으면 load_model할 때 오류 난다.
              )              # 내 생각에 keras에는 precision, recall, f1score 기능이 없는데 이게 추출되는 모델로 .h5 파일 뽑은 거니까

nb_train_samples = 800*20
nb_validation_samples = 200*20  


epochs = 10
batch_size = 32


################################# fit_generator ####################################
# steps_per_epoch : 한 번 epoch돌 때, 데이터를 몇번 볼 것인가를 정해줌(트레이닝 데이터 수/배치사이즈)
# validation_steps : 한 번 epoch 돌고 난 후, validaton set을 통해 validation accuracy를 측정할 때 validation set을 몇번 볼 것인지(validation data 수/배치사이즈)
history = model.fit_generator(
            train_generator,
            steps_per_epoch=nb_train_samples//batch_size,
            epochs=epochs,
            callbacks=callbacks)
            validation_data=validation_generator,
            validation_steps=nb_validation_samples//batch_size)
model.save('vgg_16_final.h5')
# ImageDataGenerator를 쓴 경우, fit 대신에 fit_generator를 사용하면 된다.

In [ ]:
# 출처 : https://teddylee777.github.io/tensorflow/keras-콜백함수-vol-03